## 410921202 資工四 林芷萱 midterm

In [1]:
import cv2
import numpy as np

# step1 設定起始&結束幀
frame_seq = 14 * 30
stop_frame_seq = 25 * 30

out_size = (480, 270)
cap = cv2.VideoCapture('WiiPlay.mp4')
if not cap.isOpened():
    raise IOError("Cannot open the video file")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_seq)

# step2 BGR -> HSV
def convert_to_hsv(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

def nothing(pos):
    pass

# step3 創建Window和trackbar
cv2.namedWindow('Input')
cv2.namedWindow('Test')
cv2.createTrackbar('HueMin', 'Test', 30, 180, nothing)
cv2.createTrackbar('HueMax', 'Test', 150, 180, nothing)
cv2.createTrackbar('SatMin', 'Test', 10, 255, nothing)
cv2.createTrackbar('SatMax', 'Test', 120, 255, nothing)
cv2.createTrackbar('ValMin', 'Test', 50, 255, nothing)
cv2.createTrackbar('ValMax', 'Test', 200, 255, nothing)

# step4 根據 HSV 的閾值範圍應用二值化閾值
def apply_threshold(hsv, lower, upper):
    return cv2.inRange(hsv, lower, upper)

# step5 對二值化過的影像作腐蝕和膨脹
def apply_morphological_filter(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.erode(mask, kernel)
    mask = cv2.dilate(mask, kernel)
    
    return mask

# step6-8 獲取trackbar的當前值，再根據其值的範圍對當前幀進行色彩篩選
def find_and_apply_threshold(frame, window_name):
    hsv = convert_to_hsv(frame)
    hue_min = cv2.getTrackbarPos('HueMin', 'Test')
    hue_max = cv2.getTrackbarPos('HueMax', 'Test')
    sat_min = cv2.getTrackbarPos('SatMin', 'Test')
    sat_max = cv2.getTrackbarPos('SatMax', 'Test')
    val_min = cv2.getTrackbarPos('ValMin', 'Test')
    val_max = cv2.getTrackbarPos('ValMax', 'Test')

    lower_color = np.array([hue_min, sat_min, val_min])
    upper_color = np.array([hue_max, sat_max, val_max])
    
    mask = apply_threshold(hsv, lower_color, upper_color)
    mask = apply_morphological_filter(mask)
    
    cv2.imshow(window_name, mask)

# cursor
def detect_blue_cursor(frame):
    hsv = convert_to_hsv(frame)
    lower_blue = np.array([100, 150, 50])
    upper_blue = np.array([125, 255, 255])
    blue_mask = apply_threshold(hsv, lower_blue, upper_blue)
    
    return blue_mask

# timer
def detect_yellow_timer(frame):
    hsv = convert_to_hsv(frame)
    lower_yellow = np.array([15, 200, 160])
    upper_yellow = np.array([30, 255, 255])
    yellow_mask = apply_threshold(hsv, lower_yellow, upper_yellow)
    
    return yellow_mask

# skin
def detect_human_skin(frame):
    hsv = convert_to_hsv(frame)
    lower_skin = np.array([5, 25, 50])
    upper_skin = np.array([30, 255, 255])
    
    skin_mask = apply_threshold(hsv, lower_skin, upper_skin)
    skin_mask = apply_morphological_filter(skin_mask)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    skin_mask = cv2.erode(skin_mask, kernel)
    skin_mask = cv2.dilate(skin_mask, kernel)

    return skin_mask

# step9-10
def count_and_display_skin_regions(skin_mask, window_name):
    retval, labels = cv2.connectedComponents(skin_mask)
    
    if np.max(labels) == 0:
        label_hue = np.zeros_like(labels, dtype=np.uint8)
    else:
        label_hue = np.uint8(180 * labels / np.max(labels))
        
    blank = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank, blank])
    count_display = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    count_display[label_hue == 0] = 0
    num = str(retval)
    cv2.putText(count_display, num, (0, 270), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Count and Display', count_display)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    cur_frame = cv2.resize(frame, out_size, 0, 0, interpolation=cv2.INTER_AREA)

    cv2.imshow('Input', cur_frame)

    find_and_apply_threshold(cur_frame, 'Test')

    blue_cursor_mask = detect_blue_cursor(cur_frame)
    yellow_timer_mask = detect_yellow_timer(cur_frame)
    skin_mask = detect_human_skin(cur_frame)

    cv2.imshow('Cursor', blue_cursor_mask)
    cv2.imshow('Timer', yellow_timer_mask)
    cv2.imshow('Skin', skin_mask)
    
    count_and_display_skin_regions(skin_mask, 'Count and Display')

    if cap.get(cv2.CAP_PROP_POS_FRAMES) == stop_frame_seq:
        break

    key = cv2.waitKey(1)
    if key == 32:
        cv2.waitKey()
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()


step11.Any comments regarding the midterm? Which steps you believe you have completed? Which steps bother you?  
我認為我的step1~10除了skin呈現的有點瑕疵之外，其餘都是完成的。
對我來說最困擾的部分在於調整參數，我花了很多時間調參希望能夠讓output呈現的跟demo上盡可能地相似，但也因為之前作業有做過類似的調參，對於我在medterm的調參有所幫助，對於該如呵調整比較有方向。

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

step12.Any comments regarding the classes up to now? pace too fast or slow? quiz too hard or simple? prefer C or Python  
我覺得目前的上課進度是適中的，不會太快也不會太慢，但我在一開始的課堂作業都能在課堂上完成，但到後面的幾次課堂練習就需要下課後回家完成，我覺得這是合理的，隨著課程進度的深入，會需要處理比較複雜的題目。
程式語言我比較傾向用python，因為之前專題做深度學習相關的內容也是使用python，因此python是我目前較為熟悉的語言，此外我認為電腦視覺的處理比較適合使用python。